In [ ]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    
    # Overlay
    ol = Overlay("compute_pk.bit")
    ipPK = ol.compute_public_0

    # Memory allocate
    inBufferg   = allocate(shape=(1024,), dtype=np.int8)   # ntru g
    inBufferf   = allocate(shape=(1024,), dtype=np.int8)   # ntru f
    outBufferh  = allocate(shape=(1024,), dtype=np.uint16) # pk h
    inBufferTmp = allocate(shape=(4096,), dtype=np.uint8)

    with open("g_in.txt", "r") as file1:
        line = file1.readline().split(", ")
        for i in range(len(line)):
            inBufferg[i] = int(line[i])

    with open("f_in.txt", "r") as file2:
        line = file2.readline().split(", ")
        for i in range(len(line)):
            inBufferf[i] = int(line[i])
    
    timeKernelStart = time()
    # ============= Set Kernel Argument ==============
    # control
    # 0x00 : Control signals
    #        bit 0  - ap_start (Read/Write/COH)
    #        bit 1  - ap_done (Read/COR)
    #        bit 2  - ap_idle (Read)
    # 0x10 : Data signal of ap_return
    #        bit 31~0 - ap_return[31:0] (Read) #### Return 1:True, 0:False
    # 0x18 : Data signal of h
    #        bit 31~0 - h[31:0] (Read/Write)
    # 0x1c : Data signal of h
    #        bit 31~0 - h[63:32] (Read/Write)
    # 0x20 : reserved
    # 0x24 : Data signal of f
    #        bit 31~0 - f[31:0] (Read/Write)
    # 0x28 : Data signal of f
    #        bit 31~0 - f[63:32] (Read/Write)
    # 0x2c : reserved
    # 0x30 : Data signal of g
    #        bit 31~0 - g[31:0] (Read/Write)
    # 0x34 : Data signal of g
    #        bit 31~0 - g[63:32] (Read/Write)
    # 0x38 : reserved
    # 0x3c : Data signal of logn
    #        bit 31~0 - logn[31:0] (Read/Write)
    # 0x40 : reserved
    # 0x44 : Data signal of tmp
    #        bit 31~0 - tmp[31:0] (Read/Write)
    # 0x48 : Data signal of tmp
    #        bit 31~0 - tmp[63:32] (Read/Write)
    logn = 10
    ipPK.write(0x3C, logn) # Coefficient

    # ================ Data Transfer =================
    ipPK.write(0x18, outBufferh.device_address)  # get h
    ipPK.write(0x24, inBufferf.device_address)  # write f
    ipPK.write(0x30, inBufferg.device_address)  # write g
    ipPK.write(0x44, inBufferTmp.device_address)

    # =================== Start ======================
    ipPK.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipPK.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    # ================== Compare =====================
    error_cnt = 0

    # Compare with golden value
    with open("h_out.txt", "r") as file:
        line = file.readline().split(", ")
        golden = [0 for i in range(len(line))]
        for i in range(len(line)):
            golden[i] = int(line[i])

    for l in range(len(golden)):
        if outBufferh[l] != golden[l]:
            error_cnt += 1
            print(f"expect value: {golden[l]}, actual value: {outBufferh[l]}")

    # Check the return value 
    print(f"Return value: {ipPK.read(0x10)}")
    if (ipPK.read(0x10 & 0x1) == 0):
        error_cnt += 1

    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")

    print("============================")
    print("Exit process")

